# Estimation

In this notebook, we estimate the main model in all its configurations as well as robustness checks like company panel and limiting regions

## Preliminaries
### Packages

In [1]:
lapply(c('MASS','stargazer',"lfe","ggplot2","dplyr","strucchange","gap","boot","stringr"), require, character.only = TRUE)

Loading required package: MASS

Loading required package: stargazer


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Loading required package: lfe

Loading required package: Matrix

Loading required package: ggplot2

Loading required package: dplyr


Attaching package: 'dplyr'


The following object is masked from 'package:MASS':

    select


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: strucchange

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: gap

Loading required package: gap.datasets

gap version 1.6

Loading required package: boot

Loadi

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

### Data

In [3]:
df2 <- read.csv("../data/output_data/MG2_Private.csv")
df3 <- read.csv("../data/output_data/MG3_Private.csv")

In [4]:
DF2_CC <- df2 %>%                              # Applying group_by & summarise
                  group_by(year) %>%
                  summarise(count = n_distinct(company_name))

DF3_CC <- df3 %>%                              # Applying group_by & summarise
                  group_by(year) %>%
                  summarise(count = n_distinct(company_name))

Merge data together

In [5]:
# df2 <- merge(df2,S,by = c("company_name","year"), all.x = TRUE)
# df3 <- merge(df3,S,by = c("company_name","year"), all.x = TRUE)

df2 <- merge(df2,DF2_CC,by="year",all.x=T)
df3 <- merge(df3,DF3_CC,by="year",all.x=T)

In [6]:
df2$il_and_first <- as.numeric(df2$first_il=="IL_and_FIRST")
df3$il_and_first <- as.numeric(df3$first_il=="IL_and_FIRST")

To make estimates comparable, only use "complete cases" for both Jack and Dwight.

In [7]:
df2_complete <- df2[complete.cases(df2$diff_d)&complete.cases(df2$diff_j),]
df2_reg23 <- df2_complete[(df2_complete$region=="region2")|(df2_complete$region=="region3"),]
df2_reg3 <- df2_complete[(df2_complete$region=="region3"),]
df3_complete <- df3[complete.cases(df3$diff_w),]
df3_reg23 <- df3_complete[(df3_complete$region=="region2")|(df3_complete$region=="region3"),]
df3_reg3 <- df3_complete[(df3_complete$region=="region3"),]


In [8]:
write.csv(df2_complete[,c("year","variety_id","region","location")],"MG2_ix_complete.csv")
write.csv(df3_complete[,c("year","variety_id","region","location")],"MG3_ix_complete.csv")
write.csv(df2_reg23[,c("year","variety_id","region","location")],"MG2_ix_reg23.csv")
write.csv(df3_reg23[,c("year","variety_id","region","location")],"MG3_ix_reg23.csv")
write.csv(df3_reg3[,c("year","variety_id","region","location")],"MG3_ix_reg3.csv")

### Variables and Formulas

Covariates are:
- height of private and base variety (`height` and `height_w/d/j`)
- lodging score of private and base variety (`lodging` and `lodging_w/d/j`)
- week of the year the private became mature (`maturity_week` and `maturity_week_w/d/j`) 
- week of the year the variety was planted (`planting_week` and `planting_week_w/d/j`)
- rainfall on that plot (`ppt_may/june/july/aug/sept`)
- location FE (`location`)
- seed treated or not (`seed_treated`)
- seed scn resistant or not (`scn_resist`)

Outcomes are called `diff_w/d/j_norm` and are the difference between private variety yield and Williams/Dwight/Jack planted in the same location at the same time. The average yield from 1997 is also subtracted (hence the `norm` part).

In [9]:
Y = c("diff_j_norm","diff_d_norm","diff_w_norm")

rain =  c('ppt_may', 'ppt_june', 'ppt_july', 'ppt_aug', 'ppt_sept',
             'ppt_may_sqr','ppt_june_sqr','ppt_july_sqr','ppt_aug_sqr','ppt_sept_sqr')

temp    =  c('tmin_5','tmin_6','tmin_7','tmin_8','tmin_9',
             'tmax_5','tmax_6','tmax_7','tmax_8','tmax_9',
             'tmean_5','tmean_6','tmean_7','tmean_8','tmean_9')

var_covars = c("height","lodging_comb","scn_resist","seed_treated")

factors = c("location","maturity_week","planting_week")

covars_w = c("height","lodging_comb","height_w","lodging_comb_w","seed_treated","scn_resist")
factors_w = c("location","maturity_week","maturity_week_w","planting_week")

covars_j = c("height","lodging_comb","height_j","lodging_comb_j","seed_treated","scn_resist")
factors_j = c("location","maturity_week","maturity_week_j","planting_week")

covars_d = c("height","lodging_comb","height_d","lodging_comb_d","seed_treated","scn_resist")
factors_d = c("location","maturity_week","maturity_week_d","planting_week")

Formulas are for both the `lm` object and the `felm` object.

In [10]:
fe_formula <- function(outcome,treat,X=var_covars,G=factors,C="location") {
  f <- paste0(outcome,"~",treat,"+",paste(X,collapse="+"),"|",paste(G,collapse="+"),"|","0","|",C)
  return(as.formula(f))
}
lm_formula <- function(outcome,treat,X=var_covars) {
  f <- paste0(outcome,"~",treat,"+",paste(X,collapse="+"))
  return(as.formula(f))
}

Most simple formula (`year` is the only covariate):

In [11]:
simple <- lm_formula("yield","year",X="1")

## Main Model Estimation



### Main Table (Table 2 in Paper)
##### MG 2 and MG 3 unadjusted, unadjusted with controls versus differenced, whole sample

In [12]:
out <- felm(data=df2_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp)))
DF2 <- model.frame(out)
out <- felm(data=df2_complete,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp)))
DF2b <- model.frame(out)
out <- felm(data=df3_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp)))
DF3 <- model.frame(out)
out <- felm(data=df3_complete,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp)))
DF3b <- model.frame(out)

# With covars, kg/ha
compare_gg2_x = list(lm(data=DF2,formula = "yield_kg ~first_year"),
                felm(data=df2_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_x = list(lm(data=DF3 ,formula = "yield_kg ~ first_year"),
                     felm(data=df3_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                     felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))

# With covars, bu/acr
compare_gg2_bu_x = list(lm(data=DF2b,formula = "yield ~ first_year"),
                felm(data=df2_complete,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp))),
                felm(data=df2_complete,formula = fe_formula("diff_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                felm(data=df2_complete,formula = fe_formula("diff_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_bu_x = list(lm(data=DF3b ,formula = "yield ~ first_year"),
                     felm(data=df3_complete,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp))),
                     felm(data=df3_complete,formula = fe_formula("diff_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))

# With covars and GE interaction
diff_gg_ge_kg =  list(felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year + GE + GE:first_year ",X=c(covars_d,rain,temp),G=factors_d)),
                      felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year  + GE + GE:first_year ",X=c(covars_j,rain,temp),G=factors_j)),
                      felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year  + GE + GE:first_year ",X=c(covars_w,rain,temp),G=factors_w)))

diff_gg_ge = list(felm(data=df2_complete,formula = fe_formula("diff_d","first_year  + GE + GE:first_year ",X=c(covars_d,rain,temp),G=factors_d)),
                      felm(data=df2_complete,formula = fe_formula("diff_j","first_year  + GE + GE:first_year ",X=c(covars_j,rain,temp),G=factors_j)),
                      felm(data=df3_complete,formula = fe_formula("diff_w","first_year  + GE + GE:first_year ",X=c(covars_w,rain,temp),G=factors_w)))

In [13]:
stargazer(c(compare_gg2_x),
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadj.","Unadj., Controls", "Adj. Jack", "Adj. Dwight"))
          
         # out="ex.doc")

stargazer(c(compare_gg3_x),
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadj.","Unadj., Controls","Adj. Williams"))
          


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:27:38 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{\textit{Dependent variable:}} \\ 
\cline{2-5} 
\\[-1.8ex] & Unadj. & Unadj., Controls & Adj. Jack & Adj. Dwight \\ 
\\[-1.8ex] & \textit{OLS} & \textit{felm} & \textit{felm} & \textit{felm} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 Year Trend & 47.784$^{***}$ & 29.735$^{***}$ & 25.226$^{***}$ & 26.413$^{***}$ \\ 
  & (0.697) & (7.936) & (3.534) & (3.069) \\ 
  & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 23,207 & 23,207 \\ 
Adjusted R$^{2}$ & 0.168 & 0.534 & 0.449 & 0.455 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{4}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table c

bu/acre

In [14]:
stargazer(c(compare_gg2_bu_x,compare_gg3_bu_x),
          keep="year",
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadj.","Unadj., w/ Controls", "Adj. Jack", "Adj. Dwight","Unadj.","Unadj., w/ Controls","Adj. Williams"),
          keep.stat = c("adj.rsq","n"))
          #type="text")


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:27:55 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{7}{c}{\textit{Dependent variable:}} \\ 
\cline{2-8} 
\\[-1.8ex] & Unadj. & Unadj., w/ Controls & Adj. Jack & Adj. Dwight & Unadj. & Unadj., w/ Controls & Adj. Williams \\ 
\\[-1.8ex] & \textit{OLS} & \textit{felm} & \textit{felm} & \textit{felm} & \textit{OLS} & \textit{felm} & \textit{felm} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7)\\ 
\hline \\[-1.8ex] 
 Year Trend & 0.711$^{***}$ & 0.442$^{***}$ & 0.375$^{***}$ & 0.393$^{***}$ & 0.809$^{***}$ & 0.534$^{***}$ & 0.382$^{***}$ \\ 
  & (0.010) & (0.118) & (0.053) & (0.046) & (0.010) & (0.068) & (0.061) \\ 
  & & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 23,207 & 23,207 & 27,233 & 27,2

In [15]:
stargazer(diff_gg_ge_kg,
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Year Trend X GE"),
          dep.var.labels=c("Adj. Jack","Adj. Dwight","Adj. Williams"))#,
          #type="text")


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:27:56 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & Adj. Jack & Adj. Dwight & Adj. Williams \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 13.207$^{***}$ & 13.546$^{***}$ & 15.941$^{***}$ \\ 
  & (3.231) & (3.805) & (3.551) \\ 
  & & & \\ 
 Year Trend X GE & 23.104$^{***}$ & 19.172$^{***}$ & 13.337$^{***}$ \\ 
  & (4.497) & (3.785) & (3.798) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 27,233 \\ 
Adjusted R$^{2}$ & 0.467 & 0.458 & 0.385 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


#### Different levels of covariates (Table A1)

In [17]:
compare_gg2_compare_x_j = list(felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X="1",G="0")),
                          felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=covars_j,G=c("maturity_week","maturity_week_j","planting_week"))), 
                          felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,temp,rain),G=c("maturity_week","maturity_week_j","planting_week"))),
                          felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,temp,rain),G=factors_j)))

compare_gg2_compare_x_d = list(felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X="1",G="0")),
                          felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=covars_d,G=c("maturity_week","maturity_week_d","planting_week"))), 
                          felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,temp,rain),G=c("maturity_week","maturity_week_d","planting_week"))), 
                          felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,temp,rain),G=factors_d)))

compare_gg3_compare_x_w = list(felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X="1",G="0")),
                          felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=covars_w,G=c("maturity_week","maturity_week_w","planting_week"))), 
                          felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,temp,rain),G=c("maturity_week","maturity_week_w","planting_week"))), 
                          felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,temp,rain),G=factors_w)))

In [18]:
stargazer(compare_gg2_compare_x_j,
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Year Trend X GE"),
          dep.var.labels=c(""),
          dep.var.caption = "MG II, Adj. Jack",
          add.lines=list(c("Variety Controls","","X","X","X"),
                         c("Weather Controls","","","X","X"),
                         c("Location FE","","","","X")))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:29:10 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{MG II, Adj. Jack} \\ 
\cline{2-5} 
\\[-1.8ex] & \multicolumn{4}{c}{} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 Year Trend & 40.686$^{***}$ & 40.874$^{***}$ & 27.129$^{***}$ & 25.226$^{***}$ \\ 
  & (3.519) & (3.935) & (3.524) & (3.534) \\ 
  & & & & \\ 
\hline \\[-1.8ex] 
Variety Controls &  & X & X & X \\ 
Weather Controls &  &  & X & X \\ 
Location FE &  &  &  & X \\ 
Observations & 23,441 & 23,207 & 23,207 & 23,207 \\ 
Adjusted R$^{2}$ & 0.269 & 0.319 & 0.427 & 0.449 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{4}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [19]:
stargazer(compare_gg2_compare_x_d,
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Year Trend X GE"),
          dep.var.labels=c(""),
          dep.var.caption = "MG II, Adj. Dwight",
          add.lines=list(c("Variety Controls","","X","X","X"),
                         c("Weather Controls","","","X","X"),
                         c("Location FE","","","","X")))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:29:10 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{MG II, Adj. Dwight} \\ 
\cline{2-5} 
\\[-1.8ex] & \multicolumn{4}{c}{} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 Year Trend & 45.034$^{***}$ & 34.741$^{***}$ & 28.010$^{***}$ & 26.413$^{***}$ \\ 
  & (5.018) & (5.294) & (3.001) & (3.069) \\ 
  & & & & \\ 
\hline \\[-1.8ex] 
Variety Controls &  & X & X & X \\ 
Weather Controls &  &  & X & X \\ 
Location FE &  &  &  & X \\ 
Observations & 23,441 & 23,207 & 23,207 & 23,207 \\ 
Adjusted R$^{2}$ & 0.299 & 0.348 & 0.439 & 0.455 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{4}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [20]:
stargazer(compare_gg3_compare_x_w,
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend","Year Trend X GE"),
          dep.var.labels=c(""),
          dep.var.caption = "MG III, Adj. Williams 82",
          add.lines=list(c("Variety Controls","","X","X","X"),
                         c("Weather Controls","","","X","X"),
                         c("Location FE","","","","X")))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 3:29:11 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{4}{c}{MG III, Adj. Williams 82} \\ 
\cline{2-5} 
\\[-1.8ex] & \multicolumn{4}{c}{} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 Year Trend & 31.983$^{***}$ & 22.142$^{***}$ & 26.124$^{***}$ & 25.664$^{***}$ \\ 
  & (6.490) & (4.777) & (6.161) & (4.073) \\ 
  & & & & \\ 
\hline \\[-1.8ex] 
Variety Controls &  & X & X & X \\ 
Weather Controls &  &  & X & X \\ 
Location FE &  &  &  & X \\ 
Observations & 27,565 & 27,233 & 27,233 & 27,233 \\ 
Adjusted R$^{2}$ & 0.156 & 0.228 & 0.285 & 0.375 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{4}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


### Year FE

In [21]:
out2u<-lm(data=df2_complete,formula = "yield_kg ~ relevel(as.factor(first_year),'1997')")
out2u_g<-lm(data=df2_complete,formula = "yield_kg~relevel(as.factor(first_year),'1997')*GE")

out2<-felm(data=df2_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997')",X=c(var_covars,temp,rain),G=factors))
out2_g<-felm(data=df2_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997')*GE",X=c(var_covars,temp,rain),G=factors))

outj<-felm(data=df2_complete,formula = fe_formula("diff_kg_j","relevel(as.factor(first_year),'1997')",X=c(covars_j,temp,rain),G=factors_j))
outj_g <- felm(data=df2_complete,formula = fe_formula("diff_kg_j","relevel(as.factor(first_year),'1997')*GE",X=c(covars_d,temp,rain),G=factors_d))

outd<-felm(data=df2_complete,formula = fe_formula("diff_kg_d","relevel(as.factor(first_year),'1997')",X=c(covars_d,temp,rain),G=factors_d))
outd_g<-felm(data=df2_complete,formula = fe_formula("diff_kg_d","relevel(as.factor(first_year),'1997')*GE",X=c(covars_d,temp,rain),G=factors_d))

out3u<-lm(data=df3_complete,formula = "yield_kg ~ relevel(as.factor(first_year),'1997')")
out3u_g<-lm(data=df3_complete,formula = "yield_kg~relevel(as.factor(first_year),'1997')*GE")

out3<-felm(data=df3_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997')",X=c(var_covars,temp,rain),G=factors))
out3_g<-felm(data=df3_complete,formula = fe_formula("yield_kg","relevel(as.factor(first_year),'1997')*GE",X=c(var_covars,temp,rain),G=factors))

outw<-felm(data=df3_complete,formula = fe_formula("diff_kg_w","relevel(as.factor(first_year),'1997')",X=c(covars_w,temp,rain),G=factors_w))
outw_g<-felm(data=df3_complete,formula = fe_formula("diff_kg_w","relevel(as.factor(first_year),'1997')*GE",X=c(covars_w,temp,rain),G=factors_w))

write.csv(summary(out2u)$coefficients,"../results/year_coefs2u.csv")
write.csv(summary(out2u_g)$coefficients,"../results/year_coefs2u_GE.csv")

write.csv(summary(out2)$coefficients,"../results/year_coefs2.csv")
write.csv(summary(out2_g)$coefficients,"../results/year_coefs2_GE.csv")

write.csv(summary(outj)$coefficients,"../results/year_coefs_j.csv")
write.csv(summary(outj_g)$coefficients,"../results/year_coefs_j_GE.csv")

write.csv(summary(outd)$coefficients,"../results/year_coefs_d.csv")
write.csv(summary(outd_g)$coefficients,"../results/year_coefs_d_GE.csv")

write.csv(summary(out3u)$coefficients,"../results/year_coefs3u.csv")
write.csv(summary(out3u_g)$coefficients,"../results/year_coefs3u_GE.csv")

write.csv(summary(out3)$coefficients,"../results/year_coefs3.csv")
write.csv(summary(out3_g)$coefficients,"../results/year_coefs3_GE.csv")

write.csv(summary(outw)$coefficients,"../results/year_coefs_w.csv")
write.csv(summary(outw_g)$coefficients,"../results/year_coefs_w_GE.csv")

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive de

### Variety FE

In [22]:
df2 <- read.csv("../data/output_data/MG2_Complete_FE.csv")
df3 <- read.csv("../data/output_data/MG3_Complete_FE.csv")

var_covars = c("height","lodging_comb","scn_resist","seed_treated")
covars = c(var_covars,temp,rain)
factors = c("location","maturity_week","planting_week")

#### No weather controls

In [23]:
varfe_d <- felm(data=df2,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-DWIGHT')",
                                     X=var_covars,
                                     G=factors))
varfe_j <- felm(data=df2,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-JACK')",
                                     X=var_covars,
                                     G=factors))
varfe_w <- felm(data=df3,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-WILLIAMS_82')",
                                     X=var_covars,
                                     G=factors))

coefs_d <- summary(varfe_d)$coefficients
coefs_j <- summary(varfe_j)$coefficients
coefs_w <- summary(varfe_w)$coefficients

write.csv(as.data.frame(coefs_d),"../results/dwight_fe_now.csv")
write.csv(as.data.frame(coefs_j),"../results/jack_fe_now.csv")
write.csv(as.data.frame(coefs_w),"../results/williams_fe_now.csv")

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"


#### Weather controls

In [24]:
varfe_d <- felm(data=df2,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-DWIGHT')",
                                     X=c(var_covars,rain,temp),
                                     G=factors))
varfe_j <- felm(data=df2,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-JACK')",
                                     X=c(var_covars,rain,temp),
                                     G=factors))
varfe_w <- felm(data=df3,
                formula = fe_formula("yield_kg","relevel(factor(variety_id),'public_variety-WILLIAMS_82')",
                                     X=c(var_covars,rain,temp),
                                     G=factors))

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"


In [25]:
coefs_d <- summary(varfe_d)$coefficients
coefs_j <- summary(varfe_j)$coefficients
coefs_w <- summary(varfe_w)$coefficients

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"


In [26]:
write.csv(as.data.frame(coefs_d),"../results/dwight_fe.csv")
write.csv(as.data.frame(coefs_j),"../results/jack_fe.csv")
write.csv(as.data.frame(coefs_w),"../results/williams_fe.csv")

In [27]:
coefs_d <- data.frame(coefs_d)[1:2815,]
coefs_j <- data.frame(coefs_j)[1:2815,]
coefs_w <- data.frame(coefs_w)[1:3258,]

coefs_j[,"variety_id"] <- str_extract(rownames(coefs_j),"(?<=JACK\"\\))\\w.*")
coefs_d[,"variety_id"] <- str_extract(rownames(coefs_d),"(?<=DWIGHT\"\\))\\w.*")
coefs_w[,"variety_id"] <- str_extract(rownames(coefs_w),"(?<=WILLIAMS_82\"\\))\\w.*")

var_years <- read.csv("../data/output_data/Var_Years.csv")

coefs_j <- merge(x=coefs_j,y=var_years,all.x=T,by="variety_id")
coefs_d <- merge(x=coefs_d,y=var_years,all.x=T,by="variety_id")
coefs_w <- merge(x=coefs_w,y=var_years,all.x=T,by="variety_id")

years_tested <- read.csv("../data/output_data/Years_Tested.csv")
names(years_tested) <- c("variety_id","years_tested")

coefs_j <- merge(x=coefs_j,y=years_tested,all.x=T,by="variety_id")
coefs_d <- merge(x=coefs_d,y=years_tested,all.x=T,by="variety_id")
coefs_w <- merge(x=coefs_w,y=years_tested,all.x=T,by="variety_id")


coefs_d <- coefs_d[!is.nan(coefs_d$Estimate),]
coefs_j <- coefs_j[!is.nan(coefs_j$Estimate),]
coefs_w <- coefs_w[!is.nan(coefs_w$Estimate),]

coefs_j$est <- ifelse(coefs_j[,"Pr...t.."]<=.1,coefs_j$Estimate,0)
coefs_d$est <- ifelse(coefs_d[,"Pr...t.."]<=.1,coefs_d$Estimate,0)
coefs_w$est <- ifelse(coefs_w[,"Pr...t.."]<=.1,coefs_w$Estimate,0)

coefs_j$GE <- coefs_j$GE =="True"
coefs_d$GE <- coefs_d$GE =="True"
coefs_w$GE <- coefs_w$GE =="True"

In [28]:
fe_j <- lm("est ~ first_year",coefs_j)
fe_j_fe = lm("est ~ relevel(as.factor(first_year),'1997') ",coefs_j)
fe_j_e <- lm("Estimate ~ first_year",coefs_j)
fe_j_GE <- lm("est ~ first_year*GE",coefs_j)
fe_j_m <- lm("est ~ first_year",coefs_j[coefs_j$years_tested>1,])

fe_d <- lm("est ~ first_year",coefs_d)
fe_d_fe = lm("est ~ relevel(as.factor(first_year),'1997')",coefs_d)
fe_d_e <- lm("Estimate ~ first_year",coefs_d)
fe_d_GE <- lm("est ~ first_year*GE",coefs_d)
fe_d_m <- lm("est ~ first_year",coefs_d[coefs_d$years_tested>1,])

fe_w <- lm("est ~ first_year",coefs_w)
fe_w_fe = lm("est ~ relevel(as.factor(first_year),'1997')",coefs_w)
fe_w_e <- lm("Estimate ~ first_year",coefs_w)
fe_w_GE <- lm("est ~ first_year*GE",coefs_w)
fe_w_m <- lm("est ~ first_year",coefs_w[coefs_w$years_tested>1,])

In [29]:
write.csv(summary(fe_j_fe)$coefficients,"../results/vfe_year_j.csv")
write.csv(summary(fe_d_fe)$coefficients,"../results/vfe_year_d.csv")
write.csv(summary(fe_w_fe)$coefficients,"../results/vfe_year_w.csv")

In [30]:
# With covars, kg/ha
compare_gg2_j_methods = list(felm(data=df2_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                             felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                             fe_j)

compare_gg2_d_methods = list(felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)),
                             fe_d)

compare_gg3_methods = list(felm(data=df3_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                           felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)),
                           fe_w)


In [31]:
stargazer(c(compare_gg2_j_methods),
          keep="year",
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadj.", "Adj., Jack","Variety FE, Jack"),
          keep.stat = c("adj.rsq","n"))

stargazer(c(compare_gg2_d_methods),
          keep="year",
          covariate.labels=c("Year Trend"),
          dep.var.labels=c( "Adj., Dwight","Variety FE, Dwight"),
          keep.stat = c("adj.rsq","n"))

stargazer(c(compare_gg3_methods),
          keep="year",
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadj.", "Adj., Williams 82","Variety FE, Williams 82"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:06 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & Unadj. & Adj., Jack & Variety FE, Jack \\ 
\\[-1.8ex] & \textit{felm} & \textit{felm} & \textit{OLS} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 29.735$^{***}$ & 25.226$^{***}$ & 29.404$^{***}$ \\ 
  & (7.936) & (3.534) & (1.090) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 2,914 \\ 
Adjusted R$^{2}$ & 0.534 & 0.449 & 0.200 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek

In [32]:
stargazer(fe_j,fe_j_GE,fe_d,fe_d_GE,fe_w,fe_w_GE,
          keep="year",
          covariate.labels=c("Year Trend","Year Trend X GE"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:07 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{6}{c}{NA} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Year Trend & 29.404$^{***}$ & 11.013$^{***}$ & 21.120$^{***}$ & 9.056$^{***}$ & 32.027$^{***}$ & 22.176$^{***}$ \\ 
  & (1.090) & (2.137) & (0.883) & (1.762) & (0.899) & (1.581) \\ 
  & & & & & & \\ 
 Year Trend X GE &  & 32.612$^{***}$ &  & 21.247$^{***}$ &  & 12.334$^{***}$ \\ 
  &  & (2.521) &  & (2.079) &  & (1.974) \\ 
  & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 2,914 & 2,914 & 2,915 & 2,915 & 3,379 & 3,379 \\ 
Adjusted R$^{2}$ & 0.200 & 0.272 & 0.164 & 0.211 & 0.273 & 0.288 \\ 
\hline 
\hline \\[-1

In [33]:
stargazer(fe_j,fe_j_e,fe_j_m,
          keep="year",
          covariate.labels=c("Year Trend"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:09 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & \multicolumn{3}{c}{NA} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 29.404$^{***}$ & 29.141$^{***}$ & 25.520$^{***}$ \\ 
  & (1.090) & (0.962) & (1.465) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 2,914 & 2,914 & 1,127 \\ 
Adjusted R$^{2}$ & 0.200 & 0.239 & 0.212 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [34]:
stargazer(fe_d,fe_d_e,fe_d_m,
          keep="year",
          covariate.labels=c("Year Trend"),
          keep.stat = c("adj.rsq","n"))



% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:11 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & \multicolumn{3}{c}{NA} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 21.120$^{***}$ & 29.201$^{***}$ & 19.197$^{***}$ \\ 
  & (0.883) & (0.962) & (1.188) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 2,915 & 2,915 & 1,128 \\ 
Adjusted R$^{2}$ & 0.164 & 0.240 & 0.188 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


In [35]:
stargazer(fe_w,fe_w_e,fe_w_m,
          keep="year",
          covariate.labels=c("Year Trend"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:11 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{\textit{Dependent variable:}} \\ 
\cline{2-4} 
\\[-1.8ex] & \multicolumn{3}{c}{NA} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Year Trend & 32.027$^{***}$ & 30.889$^{***}$ & 33.513$^{***}$ \\ 
  & (0.899) & (0.818) & (1.081) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Observations & 3,379 & 3,379 & 1,375 \\ 
Adjusted R$^{2}$ & 0.273 & 0.297 & 0.411 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 


### Control Variables (Tables A2-4)

In [36]:
# With covars, kg/ha
compare_gg2_x = list(felm(data=df2_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_x = list(felm(data=df3_complete,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                     felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))


In [37]:
stargazer(c(compare_gg2_x,compare_gg3_x),
          covariate.labels=c("Year Trend","Height","Lodging Score","Height (Check)","Lodging Score (Check)",
                             "Height (Check)","Lodging Score (Check)","Height (Check)","Lodging Score (Check)",
                             "SCN Resistance","Seed Treatment",
                             "Rainfall, May","Rainfall, June","Rainfall, July","Rainfall, August","Rainfall, September",
                             "Rainfall Sqrd, May","Rainfall Sqrd, June","Rainfall Sqrd, July","Rainfall Sqrd, August","Rainfall Sqrd, September",
                             "Temp Min, May","Temp Min, June","Temp Min, July","Temp Min, August","Temp Min, September",
                             "Temp Max, May","Temp Max, June","Temp Max, July","Temp Max, August","Temp Max, September",
                             "Temp Mean, May","Temp Mean, June","Temp Mean, July","Temp Mean, August","Temp Mean, September"),
          dep.var.labels=c("Unadj.", "Variety FE, Jack", "Adj., Jack", 
                                     "Variety FE, Dwight", "Adj., Dwight"),
          keep.stat = c("adj.rsq","n"))



% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:18 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{5}{c}{\textit{Dependent variable:}} \\ 
\cline{2-6} 
\\[-1.8ex] & Unadj. & Variety FE, Jack & Adj., Jack & Variety FE, Dwight & Adj., Dwight \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5)\\ 
\hline \\[-1.8ex] 
 Year Trend & 29.735$^{***}$ & 25.225$^{***}$ & 26.413$^{***}$ & 35.926$^{***}$ & 25.664$^{***}$ \\ 
  & (7.936) & (3.534) & (3.069) & (4.585) & (4.073) \\ 
  & & & & & \\ 
 Height & 19.233$^{**}$ & 4.778 & 4.810 & 17.050$^{**}$ & 7.795$^{*}$ \\ 
  & (6.205) & (3.827) & (4.491) & (7.475) & (4.084) \\ 
  & & & & & \\ 
 Lodging Score & $-$37.287 & 15.061 & 29.399 & $-$64.508 & $-$106.411$^{***}$ \\ 
  & (58.010) & (33.331) & (37.353) & (39.019) & (25.540) \\ 
  & & & & &

## Robustness Checks

### With only Region 2 and 3

In [38]:
out <- felm(data=df2_reg23,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp)))
DF2 <- model.frame(out)
out <- felm(data=df2_reg23,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp)))
DF2b <- model.frame(out)
out <- felm(data=df3_reg3,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp)))
DF3 <- model.frame(out)
out <- felm(data=df3_reg3,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp)))
DF3b <- model.frame(out)

# With covars, kg/ha
compare_gg2_x_23 = list(lm(data=DF2,formula = "yield_kg ~first_year"),
                felm(data=df2_reg23,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                felm(data=df2_reg23,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                felm(data=df2_reg23,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_x_23 = list(lm(data=DF3 ,formula = "yield_kg ~ first_year"),
                     felm(data=df3_reg3,formula = fe_formula("yield_kg","first_year",X=c(var_covars,rain,temp))),
                     felm(data=df3_reg3,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))

# With covars, bu/acr
compare_gg2_bu_x_23 = list(lm(data=DF2b,formula = "yield ~ first_year"),
                felm(data=df2_reg23,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp))),
                felm(data=df2_reg23,formula = fe_formula("diff_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                felm(data=df2_reg23,formula = fe_formula("diff_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_bu_x_23 = list(lm(data=DF3b ,formula = "yield ~ first_year"),
                     felm(data=df3_reg3,formula = fe_formula("yield","first_year",X=c(var_covars,rain,temp))),
                     felm(data=df3_reg3,formula = fe_formula("diff_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))

# With covars and GE interaction
diff_gg_ge_kg_23 =  list(felm(data=df2_reg23,formula = fe_formula("diff_kg_d","first_year + GE + GE:first_year ",X=c(covars_d,rain,temp),G=factors_d)),
                      felm(data=df2_reg23,formula = fe_formula("diff_kg_j","first_year  + GE + GE:first_year ",X=c(covars_j,rain,temp),G=factors_j)),
                      felm(data=df3_reg3,formula = fe_formula("diff_kg_w","first_year  + GE + GE:first_year ",X=c(covars_w,rain,temp),G=factors_w)))

diff_gg_ge_23 = list(felm(data=df2_reg23,formula = fe_formula("diff_d","first_year  + GE + GE:first_year ",X=c(covars_d,rain,temp),G=factors_d)),
                      felm(data=df2_reg23,formula = fe_formula("diff_j","first_year  + GE + GE:first_year ",X=c(covars_j,rain,temp),G=factors_j)),
                      felm(data=df3_reg3,formula = fe_formula("diff_w","first_year  + GE + GE:first_year ",X=c(covars_w,rain,temp),G=factors_w)))

In [39]:
stargazer(c(compare_gg2_x_23,compare_gg3_x_23),
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Unadjusted", "Adj. Jack", "Adj. Dwight","Unadjusted","Adj. Williams"))
          


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:23 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{7}{c}{\textit{Dependent variable:}} \\ 
\cline{2-8} 
\\[-1.8ex] & Unadjusted & Adj. Jack & Adj. Dwight & Unadjusted & Adj. Williams & NA & NA \\ 
\\[-1.8ex] & \textit{OLS} & \textit{felm} & \textit{felm} & \textit{felm} & \textit{OLS} & \textit{felm} & \textit{felm} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) & (7)\\ 
\hline \\[-1.8ex] 
 Year Trend & 56.628$^{***}$ & 34.827$^{***}$ & 24.528$^{***}$ & 32.414$^{***}$ & 59.614$^{***}$ & 31.376$^{***}$ & 30.365$^{**}$ \\ 
  & (0.805) & (7.618) & (4.879) & (6.764) & (0.876) & (3.047) & (6.507) \\ 
  & & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 12,305 & 12,305 & 12,305 & 12,305 & 13,799 & 13,799 & 13,799 \\ 
Adju

In [40]:
compare_mg2_23 = list(felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                        felm(data=df2_reg23,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                        felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)),
                        felm(data=df2_reg23,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)))

compare_mg3_23 = list(felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)),
                        felm(data=df3_reg3,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)))

In [41]:
stargazer(c(compare_mg2_23,compare_mg3_23),
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("Jack, Full Sample", "Jack, Region 2 and 3", "Dwight, Full Sample","Dwight, Region 2 and 3","Full Sample","Region 2 and 3"))
          


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:26 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{Jack, Full Sample} & \multicolumn{2}{c}{Jack, Region 2 and 3} & \multicolumn{2}{c}{Dwight, Full Sample} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Year Trend & 25.226$^{***}$ & 24.528$^{***}$ & 26.413$^{***}$ & 32.414$^{***}$ & 25.664$^{***}$ & 30.364$^{**}$ \\ 
  & (3.534) & (4.879) & (3.069) & (6.764) & (4.073) & (6.507) \\ 
  & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 12,305 & 23,207 & 12,305 & 27,233 & 13,799 \\ 
Adjusted R$^{2}$ & 0.449 & 0.545 & 0.455 & 0.597 & 0.375 & 0.512 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{

### Company Panel

Normal impacts

In [42]:
# Normal trend
compare_gg2_x_p = list(felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,temp,rain),G=factors_j)),
                       felm(data=df2_complete[df2_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,temp,rain),G=factors_j)),
                       felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,temp,rain),G=factors_d)),
                       felm(data=df2_complete[df2_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,temp,rain),G=factors_d)))

compare_gg3_x_p = list(felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,temp,rain),G=factors_w)),
                       felm(data=df3_complete[df3_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,temp,rain),G=factors_w)))

# GE trend
compare_gg2_x_p_ge = list(felm(data=df2_complete,formula = fe_formula("diff_kg_j","year + GE + GE:year",X=c(covars_j,temp,rain),G=factors_j)),
                       felm(data=df2_complete[df2_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_j","year + GE + GE:year",X=c(covars_j,temp,rain),G=factors_j)),
                       felm(data=df2_complete,formula = fe_formula("diff_kg_d","year + GE + GE:year",X=c(covars_d,temp,rain),G=factors_d)),
                       felm(data=df2_complete[df2_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_d","year + GE + GE:year",X=c(covars_d,temp,rain),G=factors_d)))

compare_gg3_x_p_ge = list(felm(data=df3_reg23,formula = fe_formula("diff_kg_w","year + GE + GE:year",X=c(covars_w,temp,rain),G=factors_w)),
                       felm(data=df3_complete[df3_complete$company_panel_00=="True",],formula = fe_formula("diff_kg_w","year + GE + GE:year",X=c(covars_w,temp,rain),G=factors_w)))

In [43]:
stargazer(c(compare_gg2_x_p,compare_gg3_x_p),
          keep="year",
          covariate.labels=c("Year Trend"),
          dep.var.labels=c("All Companies", "Company Panel", "All Companies", "Company Panel","All Companies", "Company Panel"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:33 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{All Companies} & \multicolumn{2}{c}{Company Panel} & \multicolumn{2}{c}{All Companies} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Year Trend & 25.226$^{***}$ & 23.829$^{***}$ & 26.413$^{***}$ & 28.508$^{***}$ & 25.664$^{***}$ & 23.162$^{***}$ \\ 
  & (3.534) & (4.574) & (3.069) & (5.119) & (4.073) & (4.059) \\ 
  & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 3,629 & 23,207 & 3,629 & 27,233 & 3,870 \\ 
Adjusted R$^{2}$ & 0.449 & 0.503 & 0.455 & 0.500 & 0.375 & 0.413 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{6}{r}{$^{*}$p$<$0.1

GE impacts

In [44]:
stargazer(c(compare_gg2_x_p_ge,compare_gg3_x_p_ge),
          keep="year",
          covariate.labels=c("Year Trend","Year Trend X GE"),
          dep.var.labels=c("All Companies", "Company Panel", "All Companies", "Company Panel","All Companies", "Company Panel"),
          keep.stat = c("adj.rsq","n"))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 4:59:34 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{All Companies} & \multicolumn{2}{c}{Company Panel} & \multicolumn{2}{c}{All Companies} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Year Trend & 21.146$^{***}$ & 24.201$^{***}$ & 19.607$^{***}$ & 26.274$^{***}$ & 23.353$^{**}$ & 23.164$^{***}$ \\ 
  & (4.103) & (4.831) & (3.609) & (4.663) & (6.697) & (5.297) \\ 
  & & & & & & \\ 
 Year Trend X GE & 17.764$^{***}$ & 5.114 & 22.343$^{***}$ & 8.936 & 17.163$^{***}$ & 6.207 \\ 
  & (4.021) & (6.491) & (4.942) & (7.765) & (4.152) & (5.678) \\ 
  & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 3,629 & 23,

### Year of trial instead of year of first trial

In [45]:
# With covars, kg/ha
compare_gg2_x_y = list(felm(data=df2_complete,formula = fe_formula("diff_kg_j","first_year",X=c(covars_j,rain,temp),G=factors_j)),
                       felm(data=df2_complete,formula = fe_formula("diff_kg_j","year",X=c(covars_j,rain,temp),G=factors_j)),
                       felm(data=df2_complete,formula = fe_formula("diff_kg_d","first_year",X=c(covars_d,rain,temp),G=factors_d)),
                       felm(data=df2_complete,formula = fe_formula("diff_kg_d","year",X=c(covars_d,rain,temp),G=factors_d)))

compare_gg3_x_y = list(felm(data=df3_complete,formula = fe_formula("diff_kg_w","first_year",X=c(covars_w,rain,temp),G=factors_w)),
                       felm(data=df3_complete,formula = fe_formula("diff_kg_w","year",X=c(covars_w,rain,temp),G=factors_w)))


In [46]:
outj_y<-felm(data=df2_complete,formula = fe_formula("diff_kg_j","relevel(as.factor(year),'1997')",X=c(covars_j,temp,rain),G=factors_j))
outj_g_y <- felm(data=df2_complete,formula = fe_formula("diff_kg_j","relevel(as.factor(year),'1997')*GE",X=c(covars_j,temp,rain),G=factors_j))

outd_y<-felm(data=df2_complete,formula = fe_formula("diff_kg_d","relevel(as.factor(year),'1997')",X=c(covars_d,temp,rain),G=factors_d))
outd_g_y<-felm(data=df2_complete,formula = fe_formula("diff_kg_d","relevel(as.factor(year),'1997')*GE",X=c(covars_d,temp,rain),G=factors_d))

outw_y<-felm(data=df3_complete,formula = fe_formula("diff_kg_w","relevel(as.factor(year),'1997')",X=c(covars_w,temp,rain),G=factors_w))
outw_g_y<-felm(data=df3_complete,formula = fe_formula("diff_kg_w","relevel(as.factor(year),'1997')*GE",X=c(covars_w,temp,rain),G=factors_w))

write.csv(summary(outj_y)$coefficients,"../results/yot_coefs_j.csv")
write.csv(summary(outj_g_y)$coefficients,"../results/yot_coefs_j_GE.csv")

write.csv(summary(outd_y)$coefficients,"../results/yot_coefs_d.csv")
write.csv(summary(outd_g_y)$coefficients,"../results/yot_coefs_d_GE.csv")

write.csv(summary(outw_y)$coefficients,"../results/yot_coefs_w.csv")
write.csv(summary(outw_g_y)$coefficients,"../results/yot_coefs_w_GE.csv")

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or not positive definite"


In [47]:
stargazer(c(compare_gg2_x_y,compare_gg3_x_y),
          keep="year",
          keep.stat = c("adj.rsq","n"),
          covariate.labels=c("Year Trend"))          


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Dec 10, 2024 - 5:00:10 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{6}{c}{\textit{Dependent variable:}} \\ 
\cline{2-7} 
\\[-1.8ex] & \multicolumn{2}{c}{diff\_kg\_j} & \multicolumn{2}{c}{diff\_kg\_d} & \multicolumn{2}{c}{diff\_kg\_w} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6)\\ 
\hline \\[-1.8ex] 
 Year Trend & 25.225$^{***}$ &  & 26.413$^{***}$ &  & 25.664$^{***}$ &  \\ 
  & (3.534) &  & (3.069) &  & (4.073) &  \\ 
  & & & & & & \\ 
 year &  & 32.769$^{***}$ &  & 33.687$^{***}$ &  & 29.896$^{***}$ \\ 
  &  & (4.606) &  & (3.910) &  & (5.158) \\ 
  & & & & & & \\ 
\hline \\[-1.8ex] 
Observations & 23,207 & 23,207 & 23,207 & 23,207 & 27,233 & 27,233 \\ 
Adjusted R$^{2}$ & 0.449 & 0.455 & 0.455 & 0.460 & 0.375 & 0.378 \\ 
\hline 
\h